# Scrape repo stats

In [ ]:
import os
os.environ['db_user']='zz3hs'
os.environ['db_pwd']='zz3hs'

In [ ]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def stars_count(url):
    html = r.get(url).text
    #soup = BeautifulSoup(html, 'lxml') 
    soup = BeautifulSoup(html, 'html.parser')
    stars_class = "social-count js-social-count"
    stars = soup.find('a', class_ = stars_class).text.strip()
    return stars

url1 = "https://github.com/tidyverse/ggplot2"
print(stars_count(url1))


html = r.get(url1).text

In [ ]:

url1 = "https://github.com/tidyverse/ggplot2"

html = r.get(url1).text

#print(html)
soup = BeautifulSoup(html, 'lxml') 

In [ ]:
myPath = os.getcwd()

print(myPath + "\n")

In [ ]:
# connect to the database, download data, limit to top 5000 repos that have the highest number of commits
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos ORDER BY commits DESC LIMIT 100'''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
raw_slug_data.head()

In [ ]:
#slugs = ["brandonleekramer/diversity", "uva-bi-sdad/oss-2020", "facebook/react"] #test data 
slugs = raw_slug_data.slug.tolist()
print(*slugs)

In [ ]:
slug1 = slugs[10]
print(slug1)
url1 =f'https://github.com/{slug1}/'
print(url1)

print(stars_count(url1))

In [ ]:
for slug in slugs:
    url = f'https://github.com/{slug}/'
    split_slugs = slug.split("/")
    login = split_slugs[0]
    repo = split_slugs[1]
    fullfilename = os.path.join(myPath, f'readme_{login}_{repo}.txt')
    urllib.request.urlretrieve(url, fullfilename)
    print(f'Finished scraping: {login}/{repo}')

In [ ]:
repo_name = []
readme_text = [] 
for filename in os.listdir(myPath):
    if filename.endswith('.txt'):
        with open(os.path.join(myPath, filename)) as f:
            content = f.read()
            soup = BeautifulSoup(content, 'html.parser')
            clean_html = ''.join(soup.article.findAll(text=True))
            repo_name.append(filename)
            readme_text.append(clean_html)
            df = pd.DataFrame({'slug': repo_name, 'readme_text': readme_text}, columns=["slug", "readme_text"])
            df['slug'] = df['slug'].str.replace('readme_','')
            df['slug'] = df['slug'].str.replace('.txt','')
            # this works because slugs can't have underscores
            df['slug'] = df['slug'].str.replace('_','/') 
df 